In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setup device-agnostic code
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/1D_1D.csv")

In [ ]:
X_train = df['x']
y_train = df['y']

In [ ]:
X_train = np.float32(X_train)
y_train = np.float32(y_train)

In [ ]:
X_train = torch.tensor(X_train)[:,None]
y_train = torch.tensor(y_train)[:,None]

In [ ]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [ ]:
class TrainDatasetFromDataFrame(Dataset):

  def __init__(self,dataframe):

    self.x_train=X_train
    self.y_train=y_train

  def __len__(self):
    return len(self.y_train)

  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]



def get_datasets_and_dataloaders(batch_size = 4):
    """
    Load the training and test datasets into data loaders.
    """
    dataframe_train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/1D_1D.csv')
    df_train = dataframe_train_data.sample(frac = 0.8)
    # Creating dataframe with the rest of the values
    df_test = dataframe_train_data.drop(df_train.index)


    train_data=TrainDatasetFromDataFrame(df_train)
    test_data=TrainDatasetFromDataFrame(df_test)

    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,  # how many samples per batch?
        shuffle=True  # shuffle the data?
    )

    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=False
    ) # don't usually need to shuffle testing data


    return train_data, test_data, train_dataloader, test_dataloader





  # Load dataloaders in global variables
TRAIN_DATASET, TEST_DATASET, TRAIN_DATALOADER, TEST_DATALOADER = get_datasets_and_dataloaders(4)

# We actually don't really need to return the datasets, but it's nice to have them for reference. If you don't,
# you can just return the dataloaders and find the datasets by calling TRAIN_DATALOADER.dataset or TEST_DATALOADER.dataset:
print(TRAIN_DATALOADER.dataset == TRAIN_DATASET)
print(TEST_DATALOADER.dataset == TEST_DATASET)

print(len(TRAIN_DATALOADER))

True
True
5000


### Création du réseau de neurones

In [ ]:
import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size = 1, hidden_size = 10, output_size = 1):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # tester différentes function d'activation
        #ELU, sigmoid, tanh
        x = self.fc1(x)
        x = nn.ELU()(x)
        x = self.fc2(x)
        x = nn.ELU()(x)
        x = self.fc3(x)
        x = nn.ELU()(x)
        x = self.fc4(x)
        x = nn.ELU()(x)
        x = self.fc5(x)
        return x

In [ ]:
net = Net(1,150,1).to(DEVICE)

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from torch.optim import lr_scheduler

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.000001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1, patience=5,
                                           threshold=0.00001, threshold_mode='abs')

n_epochs = 60000
losses = np.zeros(n_epochs)

for epoch in tqdm(range(n_epochs)):
    optimizer.zero_grad()
    x_train = X_train.to(DEVICE)
    y_train = y_train.to(DEVICE)
    #  Forward pass
    outputs = net(x_train)
    loss = torch.sqrt(criterion(outputs, y_train))
    losses[epoch] = loss
    #scheduler.step(loss)
    # Backward and optimize
    loss.backward()
    optimizer.step()

    # print the loss
    if epoch % 1000 == 0:
        print(f'Epoch [{epoch}/{n_epochs}], Loss: {loss.item():.6f}')

print('Loss (semilogy):')
print("sortie finale du modèle :", outputs)
plt.semilogy(losses)
plt.show()


  0%|          | 0/60000 [00:00<?, ?it/s]

Epoch [0/60000], Loss: 0.000205
Epoch [1000/60000], Loss: 0.000205
Epoch [2000/60000], Loss: 0.000205
Epoch [3000/60000], Loss: 0.000205
Epoch [4000/60000], Loss: 0.000205
Epoch [5000/60000], Loss: 0.000205
Epoch [6000/60000], Loss: 0.000205
Epoch [7000/60000], Loss: 0.000204
Epoch [8000/60000], Loss: 0.000204
Epoch [9000/60000], Loss: 0.000204
Epoch [10000/60000], Loss: 0.000205
Epoch [11000/60000], Loss: 0.000204
Epoch [12000/60000], Loss: 0.000204
Epoch [13000/60000], Loss: 0.000203
Epoch [14000/60000], Loss: 0.000203
Epoch [15000/60000], Loss: 0.000205
Epoch [16000/60000], Loss: 0.000203
Epoch [17000/60000], Loss: 0.000203
Epoch [18000/60000], Loss: 0.000203
Epoch [19000/60000], Loss: 0.000203
Epoch [20000/60000], Loss: 0.000202
Epoch [21000/60000], Loss: 0.000202
Epoch [22000/60000], Loss: 0.000202
Epoch [23000/60000], Loss: 0.000202
Epoch [24000/60000], Loss: 0.000205
Epoch [25000/60000], Loss: 0.000202
Epoch [26000/60000], Loss: 0.000201
Epoch [27000/60000], Loss: 0.000209
Epoch

KeyboardInterrupt: ignored

In [ ]:
torch.save(net.state_dict(), "/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/1D_100hl_ELU_loss0002.pth")

In [ ]:
net = Net(1,150,1).to(DEVICE)
net.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/1D_100hl_ELU_loss0003.pth"))

<All keys matched successfully>

## Test

In [ ]:
X = pd.read_csv("/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/1D_1D-test_set.csv")
X.head()

,Id,x
0,0,-9.379431
1,1,7.293132
2,2,-2.073387
3,3,-4.722702
4,4,1.233459


In [ ]:
X_test=X['x']
print(X_test.shape)
X_test_tensor=torch.Tensor(X_test)
X_test_tensor=torch.unsqueeze(X_test_tensor,1)
X_test_tensor = X_test_tensor.to(DEVICE)

(1000,)


In [ ]:
print(X_test_tensor.shape)

torch.Size([1000, 1])


In [ ]:
net

Net(
  (fc1): Linear(in_features=1, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=150, bias=True)
  (fc4): Linear(in_features=150, out_features=150, bias=True)
  (fc5): Linear(in_features=150, out_features=1, bias=True)
)

In [ ]:
preds=net(X_test_tensor).to(DEVICE)
preds = preds.cpu()
preds = preds.detach().numpy()
print(preds)
#print(preds)
print(type(preds))

[[-1.01409161e+00]
 [ 1.68815970e+00]
 [ 2.70980501e+00]
 [-3.90129709e+00]
 [ 2.19397354e+00]
 [ 3.47297168e+00]
 [ 3.46229315e+00]
 [ 5.51759958e-01]
 [-6.21374011e-01]
 [ 5.71671009e-01]
 [ 3.77711296e-01]
 [-2.07214689e+00]
 [ 3.42000294e+00]
 [ 3.87750196e+00]
 [ 6.80406213e-01]
 [ 1.81996918e+00]
 [ 3.33820772e+00]
 [ 3.45195031e+00]
 [ 4.50891018e+00]
 [ 4.11254358e+00]
 [-1.37130678e+00]
 [ 3.88870263e+00]
 [ 1.22382009e+00]
 [-4.72306341e-01]
 [-1.59204876e+00]
 [ 2.03354180e-01]
 [-1.57104409e+00]
 [ 4.99469948e+00]
 [ 3.50676417e+00]
 [ 3.32186747e+00]
 [-8.25695395e-01]
 [ 3.35772800e+00]
 [-2.91668415e+00]
 [ 2.22992468e+00]
 [ 4.61061430e+00]
 [-7.97956645e-01]
 [ 3.40868282e+00]
 [ 5.12483978e+00]
 [ 2.58854342e+00]
 [ 4.52213478e+00]
 [ 4.52644539e+00]
 [ 4.80932426e+00]
 [ 4.16788483e+00]
 [-3.01456022e+00]
 [ 5.06782436e+00]
 [ 3.95583200e+00]
 [-2.76549965e-01]
 [ 1.01064694e+00]
 [-6.08918428e-01]
 [ 4.71909189e+00]
 [ 2.98046875e+00]
 [-1.45433795e+00]
 [ 1.2522596

In [ ]:
index = 0
predictions = []
for i, pred in enumerate(preds):
    predictions.append({"Id":i,'Predicted':preds[i][0]})
predictions = pd.DataFrame(predictions)
print(predictions)

      Id  Predicted
0      0  -1.014092
1      1   1.688160
2      2   2.709805
3      3  -3.901297
4      4   2.193974
..   ...        ...
995  995   3.502793
996  996  -4.813044
997  997  -0.769530
998  998   2.517451
999  999   2.417130

[1000 rows x 2 columns]


In [ ]:
predictions.to_csv('/content/drive/My Drive/Colab Notebooks/DL/Kaggle1D/submission_1403.csv',index = None)